# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [1]:
import pandas as pd

In [2]:
reviews = pd.read_csv('data/reviews_sample.csv', index_col=0, parse_dates = ['date'])
recipes = pd.read_csv('data/recipes_sample.csv', parse_dates = ['submitted']).drop(['contributor_id', 'n_steps'], axis=1)
reviews[:3]

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."


In [3]:
recipes[:3]

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
reviews_5, recipes_5 = reviews.sample(frac=0.05), recipes.sample(frac=0.05)

with pd.ExcelWriter('data/recipes.xlsx') as writer:
    recipes_5.to_excel(writer, sheet_name="Рецепты")
    reviews_5.to_excel(writer, sheet_name="Отзывы")

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [2]:
import xlwings as xw

In [6]:
wb = xw.Book('data/recipes.xlsx')

sheet = wb.sheets['Рецепты']
sheet["H1"].value = 'seconds_assign'
sheet["H2:H1501"].options(index=False, header=False).value = recipes_5['minutes'] * 60

wb.save()
wb.close()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [7]:
wb = xw.Book('data/recipes.xlsx')
sht = wb.sheets['Рецепты']

formula = '=D2*60'

sht['I1'].value = 'seconds_formula'
sht['I2'].value = formula
sht['I2:I1501'].options(index=False, header=False).value = formula

wb.save()
wb.close()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [16]:
wb = xw.Book('data/recipes.xlsx')
sht = wb.sheets['Рецепты']

sht.range('H1').api.Font.Bold = True
sht.range('I1').api.Font.Bold = True

sht.range('H1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
sht.range('I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

wb.save()
wb.close()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [17]:
wb = xw.Book('data/recipes.xlsx')
sht = wb.sheets['Рецепты']

for row in sht.range('D2:D1501'):
    cell_range = sht.range('D' + str(row.row))
    minutes = cell_range.value
        
    if minutes < 5:
        cell_range.colorindex = 4 # green
    elif minutes < 10:
        cell_range.colorindex = 6 # yellow
    else:
        cell_range.colorindex = 3 # red

wb.save()
wb.close()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
wb = xw.Book('data/recipes.xlsx')
recipes_sht = wb.sheets['Рецепты']

recipe_ids = recipes_sht.range('C2:C1501').value

recipes_sht.range('J1').value = 'n_reviews'

# заполняем столбец n_reviews с помощью формул Excel
for i, recipe_id in enumerate(recipe_ids):
    formula = f'=COUNTIF(Отзывы!$C:$C, C{i+2})'
    recipes_sht.range(f'J{i+2}').formula = formula

# сохраняем и закрываем файл
wb.save()
wb.close()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    wb = xw.Book('data/recipes.xlsx')
    recipes_sht = wb.sheets['Рецепты']
    reviews_sht = wb.sheets['Отзывы']

    recipe_ratings = {int(id_cell.value): float(rating_cell.value) for id_cell, rating_cell in zip(recipes_sht.range('C2:C1501'), recipes_sht.range('E2:E100'))}
    recipe_ids = [int(id_cell.value) for id_cell in recipes_sht.range('C2:C1501').cells if id_cell.value]

    for row in reviews_sht.range('C2').expand('down').rows:
        rating = float(row.cells[4].value)

        if not 0 <= rating <= 5:
            row.color = (255, 0, 0)

        recipe_id = int(row.cells[0].value)
        if recipe_id not in recipe_ids:
            row.color = (255, 0, 0)
    print(rating)

validate()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [2]:
import csv
import pandas as pd
import xlwings as xw

# считываем данные из csv файла и преобразуем их в DataFrame
df = pd.read_csv("data/recipes_model.csv")

# создаем новый Excel файл и новый лист в этом файле
wb = xw.Book()
ws = wb.sheets.add('Модель')

# записываем данные начиная с ячейки A2
cell_range = ws.range('A2').options(index=False, header=False)
cell_range.value = df.values.tolist()

# сохраняем файл
wb.save('data/recipes_model.xlsx')

com_error: (-2147352567, 'Ошибка.', (0, 'Microsoft Excel', 'Эта команда недоступна, так как истек срок действия лицензии на данное приложение.', 'xlmain11.chm', 0, -2146827284), None)

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
wb = xw.Book('data/recipes_model.xlsx')
ws = wb.sheets['Модель']

last_row = ws.cells.last_cell.row

for i in range(2, last_row + 1):
    values_b_c = f'{ws.range(f"B{i}").value} {ws.range(f"C{i}").value.upper()}'
    words_k = ws.range(f"K{i}").value.split()
    if "PK" in words_k:
        keys = "PRIMARY KEY"
    elif "FK" in words_k:
        table_name = ws.range(f"H{i}").value
        column_name = ws.range(f"I{i}").value
        keys = f"REFERENCES {table_name}({column_name})"
    else:
        keys = ""
    is_required = ws.range(f"L{i}").value == "Y" and "PK" not in words_k
    not_null = "NOT NULL" if is_required else ""
    formula = f'{values_b_c} {" ".join(words_k)} {keys} {not_null}'
    ws.range(f"J{i}").formula = formula

wb.save()

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
wb = xw.Book('recipes_model.xlsx')
ws = wb.sheets['Модель']

header_format = ws.range("A2").expand("table").rows[0].api.Interior
header_format.Color = 0x00ccff

ws.autofit()

header_font = ws.range("A2").expand("table").rows[0].api.Font
header_font.Bold = True

ws.autofilter.apply()

wb.save()